In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import re
import warnings
from typing import List, Dict, Union, Optional
import textwrap
from dataclasses import dataclass

In [1]:
# =============================================================================
# DOCUMENT SUMMARIZER USING HUGGING FACE PRE-TRAINED MODELS
# Complete toolkit for text summarization
# =============================================================================


# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

print("="*60)
print("DOCUMENT SUMMARIZER - HUGGING FACE MODELS")
print("="*60)

# =============================================================================
# MODEL CONFIGURATIONS
# =============================================================================

@dataclass
class ModelConfig:
    """Configuration for different summarization models"""
    name: str
    model_id: str
    max_input_length: int
    description: str

# Available pre-trained models
AVAILABLE_MODELS = {
    "led": ModelConfig(
        name="LED",
        model_id="allenai/led-base-16384",
        max_input_length=16384,
        description="Great for long documents, supports up to 16,384 tokens"
    ),
    "bart": ModelConfig(
        name="BART",
        model_id="facebook/bart-large",
        max_input_length=1024,
        description="Excellent for general summarization, up to 1024 tokens"
    ),
    "t5": ModelConfig(
        name="T5",
        model_id="t5-small",
        max_input_length=512,
        description="Versatile model, good for various text types (faster but less accurate)"
    ),
    "t5_base": ModelConfig(
        name="T5-Base",
        model_id="t5-base",
        max_input_length=512,
        description="Better quality than t5-small, still relatively fast"
    ),
    "pegasus": ModelConfig(
        name="Pegasus",
        model_id="google/pegasus-cnn_dailymail",
        max_input_length=1024,
        description="Specialized for news summarization, very high quality"
    ),
    "flan_t5": ModelConfig(
        name="Flan-T5",
        model_id="google/flan-t5-small",
        max_input_length=512,
        description="Instruction-tuned T5, follows instructions better"
    )
}

def list_available_models():
    """Display available models"""
    print("Available Summarization Models:")
    print("-" * 50)
    for key, config in AVAILABLE_MODELS.items():
        print(f"🤖 {config.name} ({key})")
        print(f"   Model: {config.model_id}")
        print(f"   Max Length: {config.max_input_length} tokens")
        print(f"   Description: {config.description}")
        print()

list_available_models()

# =============================================================================
# SIMPLE SUMMARIZER CLASS
# =============================================================================

class DocumentSummarizer:
    """
    Simple document summarizer using Hugging Face models
    """
    
    def __init__(self, model_name: str = "bart", device: str = "auto"):
        """
        Initialize the summarizer
        
        Args:
            model_name: Model to use ('bart', 't5', 'pegasus', etc.)
            device: Device to use ('auto', 'cpu', 'cuda')
        """
        self.model_name = model_name
        self.config = AVAILABLE_MODELS.get(model_name, AVAILABLE_MODELS["led"])
        
        print(f"Loading {self.config.name} model...")
        print(f"Model ID: {self.config.model_id}")
        
        # Initialize the summarization pipeline
        self.summarizer = pipeline(
            "summarization",
            model=self.config.model_id,
            tokenizer=self.config.model_id,
            device=0 if device == "auto" and torch.cuda.is_available() else -1
        )
        
        print(f"✅ {self.config.name} model loaded successfully!")
        print(f"📱 Using device: {'GPU' if self.summarizer.device.type == 'cuda' else 'CPU'}")
    
    def summarize(self, 
                  text: str, 
                  max_length: int = 150, 
                  min_length: int = 50,
                  do_sample: bool = False) -> str:
        """
        Summarize a single document
        
        Args:
            text: Input text to summarize
            max_length: Maximum length of summary
            min_length: Minimum length of summary
            do_sample: Whether to use sampling (more creative but less consistent)
            
        Returns:
            Generated summary
        """
        # Handle special case for T5 models (need "summarize:" prefix)
        if "t5" in self.config.model_id.lower():
            text = f"summarize: {text}"
        
        try:
            # Generate summary
            summary = self.summarizer(
                text,
                max_length=max_length,
                min_length=min_length,
                do_sample=do_sample,
                truncation=True
            )
            
            return summary[0]['summary_text']
            
        except Exception as e:
            return f"Error generating summary: {str(e)}"
    
    def summarize_long_document(self, 
                               text: str, 
                               chunk_size: int = None,
                               max_length: int = 150,
                               min_length: int = 50) -> str:
        """
        Summarize long documents by chunking
        
        Args:
            text: Long input text
            chunk_size: Size of each chunk (auto-calculated if None)
            max_length: Maximum length of each chunk summary
            min_length: Minimum length of each chunk summary
            
        Returns:
            Combined summary
        """
        if chunk_size is None:
            chunk_size = self.config.max_input_length - 100  # Leave some buffer
        
        # Split text into chunks
        chunks = self._split_text_into_chunks(text, chunk_size)
        
        print(f"📄 Document split into {len(chunks)} chunks")
        
        # Summarize each chunk
        chunk_summaries = []
        for i, chunk in enumerate(chunks):
            print(f"Summarizing chunk {i+1}/{len(chunks)}...")
            summary = self.summarize(chunk, max_length, min_length)
            chunk_summaries.append(summary)
        
        # Combine summaries
        combined_summary = " ".join(chunk_summaries)
        
        # If combined summary is still too long, summarize it again
        if len(combined_summary.split()) > max_length:
            print("🔄 Combined summary too long, creating final summary...")
            final_summary = self.summarize(combined_summary, max_length, min_length)
            return final_summary
        
        return combined_summary
    
    def _split_text_into_chunks(self, text: str, chunk_size: int) -> List[str]:
        """Split text into chunks of approximately chunk_size tokens"""
        # Simple word-based chunking (rough approximation)
        words = text.split()
        chunks = []
        
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i:i + chunk_size])
            chunks.append(chunk)
        
        return chunks
    
    def batch_summarize(self, 
                       texts: List[str], 
                       max_length: int = 150,
                       min_length: int = 50) -> List[str]:
        """
        Summarize multiple documents
        
        Args:
            texts: List of input texts
            max_length: Maximum length of summaries
            min_length: Minimum length of summaries
            
        Returns:
            List of summaries
        """
        summaries = []
        
        for i, text in enumerate(texts):
            print(f"📝 Summarizing document {i+1}/{len(texts)}")
            summary = self.summarize(text, max_length, min_length)
            summaries.append(summary)
        
        return summaries

# =============================================================================
# USAGE EXAMPLES
# =============================================================================

print("\n" + "="*60)
print("USAGE EXAMPLES")
print("="*60)

# Sample documents for testing
sample_documents = {
    "news_article": """
    Artificial intelligence (AI) is rapidly transforming industries across the globe, from healthcare and finance to transportation and entertainment. Machine learning algorithms are becoming increasingly sophisticated, enabling computers to perform tasks that were once thought to require human intelligence. Companies are investing billions of dollars in AI research and development, hoping to gain a competitive edge in the digital economy. However, the rapid advancement of AI also raises important questions about job displacement, privacy, and ethical considerations. Experts warn that society must carefully consider the implications of widespread AI adoption to ensure that the benefits are shared equitably and that potential risks are properly managed. The future of AI will likely depend on how well we can balance innovation with responsibility, creating systems that enhance human capabilities rather than replace them entirely.
    """,
    
    "technical_article": """
    Deep learning neural networks have revolutionized the field of computer vision in recent years. Convolutional Neural Networks (CNNs) have proven particularly effective for image recognition tasks, achieving superhuman performance on various benchmarks. The architecture of CNNs is inspired by the visual cortex of animals, using layers of convolution and pooling operations to extract hierarchical features from images. Transfer learning has become a popular technique, allowing practitioners to leverage pre-trained models and fine-tune them for specific tasks with limited data. Popular frameworks like TensorFlow and PyTorch have made deep learning more accessible to researchers and developers. Recent advances include attention mechanisms, which help models focus on relevant parts of the input, and transformer architectures, which have shown remarkable success in both natural language processing and computer vision tasks. The field continues to evolve rapidly, with new architectures and training techniques being developed regularly.
    """,
    
    "business_report": """
    The global e-commerce market experienced unprecedented growth during 2020-2023, driven primarily by the COVID-19 pandemic and changing consumer behavior. Online retail sales increased by over 200% in many categories, with companies like Amazon, Shopify, and Alibaba reporting record profits. Small businesses rapidly adopted digital transformation strategies, moving their operations online to survive lockdowns and social distancing measures. Mobile commerce (m-commerce) emerged as a dominant trend, with over 70% of online purchases now made through smartphones and tablets. Social media platforms integrated shopping features, creating new opportunities for direct-to-consumer brands. Supply chain disruptions highlighted the importance of diversified logistics networks and local fulfillment centers. Looking ahead, experts predict that hybrid shopping experiences combining online and offline elements will become the new standard, as consumers increasingly expect seamless omnichannel experiences across all touchpoints.
    """
}

# =============================================================================
# EXAMPLE 1: BASIC SUMMARIZATION
# =============================================================================

print("Example 1: Basic Document Summarization")
print("-" * 50)

# Initialize summarizer (you can change the model here)
summarizer = DocumentSummarizer(model_name="bart")  

# Summarize a news article
news_text = sample_documents["news_article"]
print("📄 Original Text:")
print(textwrap.fill(news_text.strip(), width=80))
print(f"\n📊 Original length: {len(news_text.split())} words")

summary = summarizer.summarize(
    news_text,
    max_length=79,
    min_length=30
)

print(f"\n📝 Summary:")
print(textwrap.fill(summary, width=80))
print(f"📊 Summary length: {len(summary.split())} words")

# =============================================================================
# EXAMPLE 2: LONG DOCUMENT SUMMARIZATION
# =============================================================================

print("\n" + "="*60)
print("Example 2: Long Document Summarization")
print("-" * 50)

# Create a longer document by combining samples
long_document = "\n\n".join(sample_documents.values())
print(f"📄 Long document length: {len(long_document.split())} words")

long_summary = summarizer.summarize_long_document(
    long_document,
    max_length=179,
    min_length=80
)

print(f"\n📝 Long Document Summary:")
print(textwrap.fill(long_summary, width=80))
print(f"📊 Summary length: {len(long_summary.split())} words")

# =============================================================================
# EXAMPLE 3: BATCH SUMMARIZATION
# =============================================================================

print("\n" + "="*60)
print("Example 3: Batch Summarization")
print("-" * 50)

# Summarize multiple documents
document_list = list(sample_documents.values())
batch_summaries = summarizer.batch_summarize(
    document_list,
    max_length=80,
    min_length=30
)

for i, (doc_type, summary) in enumerate(zip(sample_documents.keys(), batch_summaries)):
    print(f"\n📄 {doc_type.replace('_', ' ').title()}:")
    print(textwrap.fill(summary, width=80))

# =============================================================================
# EXAMPLE 4: DIFFERENT MODELS COMPARISON
# =============================================================================

print("\n" + "="*60)
print("Example 4: Model Comparison")
print("-" * 50)

def compare_models(text: str, models: List[str] = ["led", "t5"]):
    """Compare summaries from different models"""
    results = {}
    
    for model_name in models:
        try:
            print(f"🤖 Testing {model_name.upper()} model...")
            temp_summarizer = DocumentSummarizer(model_name=model_name)
            summary = temp_summarizer.summarize(text, max_length=100, min_length=30)
            results[model_name] = summary
            # Clean up to save memory
            del temp_summarizer
        except Exception as e:
            results[model_name] = f"Error: {str(e)}"
    
    return results

# Compare models on the same text
comparison_text = sample_documents["technical_article"]
model_comparisons = compare_models(comparison_text, ["bart", "t5"])

print(f"\n📄 Original Text (first 200 chars):")
print(comparison_text[:200] + "...")

for model, summary in model_comparisons.items():
    print(f"\n🤖 {model.upper()} Summary:")
    print(textwrap.fill(summary, width=80))

# =============================================================================
# EXAMPLE 5: ADVANCED FEATURES
# =============================================================================

print("\n" + "="*60)
print("Example 5: Advanced Summarization Features")
print("-" * 50)

class AdvancedSummarizer(DocumentSummarizer):
    """Advanced summarizer with additional features"""
    
    def extractive_keywords(self, text: str, top_k: int = 10) -> List[str]:
        """Extract keywords from text (simple frequency-based)"""
        # Remove common stop words
        stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those'}
        
        # Simple keyword extraction
        words = re.findall(r'\b[a-zA-Z]{4,}\b', text.lower())
        word_freq = {}
        
        for word in words:
            if word not in stop_words:
                word_freq[word] = word_freq.get(word, 0) + 1
        
        # Return top keywords
        sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
        return [word for word, freq in sorted_words[:top_k]]
    
    def analyze_document(self, text: str) -> Dict[str, Union[str, List[str], int]]:
        """Comprehensive document analysis"""
        return {
            "summary": self.summarize(text),
            "keywords": self.extractive_keywords(text),
            "word_count": len(text.split()),
            "sentence_count": len(re.split(r'[.!?]+', text)),
            "estimated_reading_time": f"{len(text.split()) // 200 + 1} minutes"
        }

# Demonstrate advanced features
advanced_summarizer = AdvancedSummarizer(model_name="LED")
analysis = advanced_summarizer.analyze_document(sample_documents["business_report"])

print("📊 Document Analysis:")
print(f"📝 Summary: {textwrap.fill(analysis['summary'], width=70)}")
print(f"🔑 Keywords: {', '.join(analysis['keywords'])}")
print(f"📊 Word Count: {analysis['word_count']}")
print(f"📊 Sentences: {analysis['sentence_count']}")
print(f"⏱️ Reading Time: {analysis['estimated_reading_time']}")

# =============================================================================
# EXAMPLE 6: CUSTOM DOCUMENT PROCESSOR
# =============================================================================

print("\n" + "="*60)
print("Example 6: Custom Document Processor")
print("-" * 50)

def process_your_document(file_path_or_text: str, 
                         model_name: str = "bart",
                         summary_length: str = "medium") -> Dict[str, str]:
    """
    Process your own document
    
    Args:
        file_path_or_text: Path to text file or direct text input
        model_name: Model to use for summarization
        summary_length: "short", "medium", or "long"
        
    Returns:
        Dictionary with summary and metadata
    """
    
    # Length configurations
    length_configs = {
        "short": {"max_length": 80, "min_length": 20},
        "medium": {"max_length": 150, "min_length": 50},
        "long": {"max_length": 300, "min_length": 100}
    }
    
    config = length_configs.get(summary_length, length_configs["medium"])
    
    # Determine if input is file path or text
    if len(file_path_or_text) < 1000 and (file_path_or_text.endswith('.txt') or '/' in file_path_or_text or '\\' in file_path_or_text):
        try:
            with open(file_path_or_text, 'r', encoding='utf-8') as file:
                text = file.read()
            source = f"File: {file_path_or_text}"
        except:
            text = file_path_or_text
            source = "Direct input"
    else:
        text = file_path_or_text
        source = "Direct input"
    
    # Initialize summarizer
    summarizer = DocumentSummarizer(model_name=model_name)
    
    # Choose summarization method based on text length
    if len(text.split()) > 1000:
        summary = summarizer.summarize_long_document(text, **config)
    else:
        summary = summarizer.summarize(text, **config)
    
    return {
        "source": source,
        "original_length": f"{len(text.split())} words",
        "summary": summary,
        "summary_length": f"{len(summary.split())} words",
        "compression_ratio": f"{len(summary.split()) / len(text.split()) * 100:.1f}%"
    }

# Example usage
print("📄 Processing Custom Document:")

# You can replace this with your own text or file path
your_text = """
Machine learning is a subset of artificial intelligence that focuses on the development of algorithms and statistical models that enable computer systems to improve their performance on a specific task through experience, without being explicitly programmed. The core idea behind machine learning is to create systems that can automatically learn and improve from data, identifying patterns and making predictions or decisions based on input information. There are three main types of machine learning: supervised learning, where algorithms learn from labeled training data; unsupervised learning, where systems find hidden patterns in data without labeled examples; and reinforcement learning, where agents learn through interaction with an environment by receiving rewards or penalties for their actions. Machine learning has applications across numerous fields including computer vision, natural language processing, recommendation systems, fraud detection, medical diagnosis, and autonomous vehicles. As data becomes increasingly abundant and computational power continues to grow, machine learning techniques are becoming more sophisticated and widely adopted across industries.
"""

result = process_your_document(
    your_text,
    model_name="bart",
    summary_length="medium"
)

for key, value in result.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

# =============================================================================
# INSTALLATION AND SETUP INSTRUCTIONS
# =============================================================================

print("\n" + "="*60)
print("INSTALLATION AND SETUP INSTRUCTIONS")
print("="*60)

setup_instructions = """
📦 REQUIRED PACKAGES:
   pip install transformers torch torchvision torchaudio
   pip install accelerate  # For faster loading
   pip install sentencepiece  # For T5 models

🚀 QUICK START:
   1. Copy this code to your Python file/Jupyter notebook
   2. Install required packages
   3. Run the examples above
   4. Replace sample text with your own documents

💡 TIPS FOR BETTER PERFORMANCE:
   • Use GPU if available (automatically detected)
   • Start with smaller models (t5-small) for testing
   • Use BART for general text, Pegasus for news
   • Chunk long documents for better quality
   • Experiment with max_length and min_length

🎯 RECOMMENDED MODELS BY USE CASE:
   • News Articles: facebook/bart-large-cnn or google/pegasus-cnn_dailymail
   • Academic Papers: t5-base or google/flan-t5-base  
   • General Text: facebook/bart-large-cnn
   • Conversations: philschmid/bart-large-cnn-samsum
   • Fast Processing: t5-small (lower quality but faster)

⚡ PERFORMANCE NOTES:
   • First run downloads models (~500MB-2GB)
   • GPU significantly speeds up processing
   • Models are cached after first download
   • Batch processing is more efficient for multiple documents
"""

print(setup_instructions)

print("\n✅ Ready to use! Try replacing the sample documents with your own text.")
print("🎯 Start with: summarizer = DocumentSummarizer('bart')")
print("📝 Then use: summary = summarizer.summarize('Your text here')")

DOCUMENT SUMMARIZER - HUGGING FACE MODELS
Available Summarization Models:
--------------------------------------------------
🤖 LED (led)
   Model: allenai/led-base-16384
   Max Length: 16384 tokens
   Description: Great for long documents, supports up to 16,384 tokens

🤖 BART (bart)
   Model: facebook/bart-large
   Max Length: 1024 tokens
   Description: Excellent for general summarization, up to 1024 tokens

🤖 T5 (t5)
   Model: t5-small
   Max Length: 512 tokens
   Description: Versatile model, good for various text types (faster but less accurate)

🤖 T5-Base (t5_base)
   Model: t5-base
   Max Length: 512 tokens
   Description: Better quality than t5-small, still relatively fast

🤖 Pegasus (pegasus)
   Model: google/pegasus-cnn_dailymail
   Max Length: 1024 tokens
   Description: Specialized for news summarization, very high quality

🤖 Flan-T5 (flan_t5)
   Model: google/flan-t5-small
   Max Length: 512 tokens
   Description: Instruction-tuned T5, follows instructions better


USAGE EX

Device set to use cpu


✅ BART model loaded successfully!
📱 Using device: CPU
📄 Original Text:
Artificial intelligence (AI) is rapidly transforming industries across the
globe, from healthcare and finance to transportation and entertainment. Machine
learning algorithms are becoming increasingly sophisticated, enabling computers
to perform tasks that were once thought to require human intelligence. Companies
are investing billions of dollars in AI research and development, hoping to gain
a competitive edge in the digital economy. However, the rapid advancement of AI
also raises important questions about job displacement, privacy, and ethical
considerations. Experts warn that society must carefully consider the
implications of widespread AI adoption to ensure that the benefits are shared
equitably and that potential risks are properly managed. The future of AI will
likely depend on how well we can balance innovation with responsibility,
creating systems that enhance human capabilities rather than replace them
e

Device set to use cpu


✅ BART model loaded successfully!
📱 Using device: CPU
🤖 Testing T5 model...
Loading T5 model...
Model ID: t5-small


...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu
Input ids are automatically padded from 184 to 1024 to be a multiple of `config.attention_window`: 1024


✅ LED model loaded successfully!
📱 Using device: CPU
📊 Document Analysis:
📝 Summary: 
🔑 Keywords: online, commerce, consumer, over, social, shopping, experiences, global, market, experienced
📊 Word Count: 131
📊 Sentences: 8
⏱️ Reading Time: 1 minutes

Example 6: Custom Document Processor
--------------------------------------------------
📄 Processing Custom Document:
Loading BART model...
Model ID: facebook/bart-large


Device set to use cpu


✅ BART model loaded successfully!
📱 Using device: CPU
Source: Direct input
Original Length: 158 words
Summary: Machine learning is a subset of artificial intelligence that focuses on the development of algorithms and statistical models that enable computer systems to improve their performance on a specific task through experience, without being explicitly programmed. The core idea behind machine learning is to create systems that can automatically learn and improve from data, identifying patterns and making predictions or decisions based on input information. As data becomes increasingly abundant and computational power continues to grow, machine learning techniques are becoming more sophisticated and widely adopted across industries. Machine learning has applications across numerous fields including computer vision, natural language processing, recommendation systems, fraud detection, medical diagnosis, and autonomous vehicles. There are three main types of machine learning: supervise

In [2]:
your_text = """
Non-Linearity as "Bending the Line"

Straight Line (Linear): A straight line on a graph represents a linear relationship, where the output changes proportionally with the input (e.g., <math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mi>y</mi><mo>=</mo><mn>2</mn><mi>x</mi><mo>+</mo><mn>3</mn></mrow><annotation encoding="application/x-tex"> y = 2x + 3 </annotation></semantics></math>). This is what a linear activation function (<math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mi>f</mi><mo stretchy="false">(</mo><mi>x</mi><mo stretchy="false">)</mo><mo>=</mo><mi>x</mi></mrow><annotation encoding="application/x-tex"> f(x) = x </annotation></semantics></math>) produces. However, real-world data points often don’t fall neatly along a straight line—think of patterns like circles, curves, or scattered clusters.
Bending the Line (Non-Linear): Non-linear activation functions (like ReLU, sigmoid, or tanh) allow the model to "bend" or transform the decision boundary or function to fit complex patterns. This bending helps the model cover or classify data points that don’t follow a straight-line relationship, such as curves, thresholds, or irregular shapes.

Why Bending is Needed
Imagine you’re plotting data points on a graph and trying to separate or predict them:

If the points form a circular pattern (e.g., points inside vs. outside a circle), a straight line can’t separate them effectively.
A non-linear function, enabled by activation functions like ReLU, allows the model to create a curved or piecewise boundary that can "wrap around" or better fit those points.

This bending is what allows neural networks to solve complex problems like image recognition or speech processing, where relationships between inputs (e.g., pixels, audio signals) and outputs (e.g., "cat" or "dog") are highly non-linear.
How Non-Linearity Works in Neural Networks

Each neuron in a neural network computes a linear transformation (<math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mi>z</mi><mo>=</mo><mi>W</mi><mi>x</mi><mo>+</mo><mi>b</mi></mrow><annotation encoding="application/x-tex"> z = Wx + b </annotation></semantics></math>), which, if left alone, would produce a straight-line relationship.
A non-linear activation function (e.g., ReLU: <math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mi>f</mi><mo stretchy="false">(</mo><mi>x</mi><mo stretchy="false">)</mo><mo>=</mo><mi>max</mi><mo>⁡</mo><mo stretchy="false">(</mo><mn>0</mn><mo separator="true">,</mo><mi>x</mi><mo stretchy="false">)</mo></mrow><annotation encoding="application/x-tex"> f(x) = \max(0, x) </annotation></semantics></math>) is applied to this output, introducing a "bend" or change in behavior (e.g., ReLU sets negative values to 0, creating a kink).
When multiple layers with non-linear activations are stacked, these bends combine to form complex curves or surfaces that can fit intricate data patterns.

Visualizing the Idea
To illustrate your analogy of bending the line, let’s consider a simple example. Suppose you have data points that form a parabolic pattern (e.g., <math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mi>y</mi><mo>=</mo><msup><mi>x</mi><mn>2</mn></msup></mrow><annotation encoding="application/x-tex"> y = x^2 </annotation></semantics></math>) rather than a straight line. Below is a chart comparing:

A linear model (straight line) that fails to fit the data well.
A non-linear model (enabled by something like ReLU in a neural network) that bends to fit the parabolic pattern.
"""

result = process_your_document(
    your_text,
    model_name="bart",
    summary_length="medium"
)

for key, value in result.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

Loading BART model...
Model ID: facebook/bart-large


Device set to use cpu


✅ BART model loaded successfully!
📱 Using device: CPU
Source: Direct input
Original Length: 399 words
Summary: How Non-Linearity Works in Neural Networks (Neural Networks) (NBNs) is a way for neural networks to "bend" or transform the decision boundary or function to fit complex patterns. This bending helps the model cover or classify data points that don’t follow a straight-line relationship, such as curves, thresholds, or irregular shapes. This is what a linear activation function (<math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mi>y</mi><mo>=</mo><min>2</mn><mi><x</mi>mo>+</mo></mn>3</mn></m
Summary Length: 60 words
Compression Ratio: 15.0%
